# Bound on Global Extremum

**Adapted from**: SOSTOOLS' SOSDEMO3 (See Section 4.3 of [SOSTOOLS User's Manual](http://sysos.eng.ox.ac.uk/sostools/sostools.pdf))

In [1]:
using DynamicPolynomials
@polyvar x1 x2

(x1, x2)

The Goldstein-Price function $f(x)$ is defined as follows:

In [2]:
f1 = x1 + x2 + 1
f2 = 19 - 14x1 + 3x1^2 - 14x2 + 6x1*x2 + 3x2^2
f3 = 2x1 - 3x2
f4 = 18 - 32x1 + 12x1^2 + 48x2 - 36x1*x2 + 27x2^2
f = (1 + f1^2 * f2) * (30 + f3^2 * f4)

144x1⁸ - 288x1⁷x2 - 648x1⁶x2² + 1224x1⁵x2³ + 1305x1⁴x2⁴ - 1836x1³x2⁵ - 1458x1²x2⁶ + 972x1x2⁷ + 729x2⁸ - 768x1⁷ + 1344x1⁶x2 + 2592x1⁵x2² - 4080x1⁴x2³ - 3480x1³x2⁴ + 3672x1²x2⁵ + 1944x1x2⁶ - 648x2⁷ + 952x1⁶ - 168x1⁵x2 - 5370x1⁴x2² + 1240x1³x2³ + 8730x1²x2⁴ - 1188x1x2⁵ - 4428x2⁶ + 1344x1⁵ - 7680x1⁴x2 + 9840x1³x2² + 5040x1²x2³ - 11880x1x2⁴ + 1944x2⁵ - 2454x1⁴ + 5784x1³x2 + 7776x1²x2² - 23616x1x2³ + 14346x2⁴ - 1072x1³ + 7344x1²x2 - 19296x1x2² + 12288x2³ + 1260x1² - 4680x1x2 + 3060x2² + 720x1 + 720x2 + 600

We need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v0.21.6/installation/#Supported-solvers) for a list of the available choices.
We use `SOSModel` instead of `Model` to be able to use the `>=` syntax for Sum-of-Squares constraints.

In [3]:
using SumOfSquares
using CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
model = SOSModel(solver);

We create the decision variable $\gamma$ that will be the lower bound to the Goldstein-Price function.
We maximize it to have the highest possible lower bound.

In [4]:
@variable(model, γ)
@objective(model, Max, γ)

γ

We constrain $\gamma$ to be a lower bound with the following constraint
that ensures that $f(x_1, x_2) \ge \gamma$ for all $x_1, x_2$.

In [5]:
@constraint(model, f >= γ)

JuMP.optimize!(model)

We verify that the solver has found a feasible solution:

In [6]:
JuMP.primal_status(model)

NEARLY_FEASIBLE_POINT::ResultStatusCode = 2

We can now obtain the lower bound either with `value(γ)` or `objective_value(model)`:

In [7]:
objective_value(model)

2.9999663989828473

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*